In [1]:
import numpy as np
import csv
from mulearn.kernel import PrecomputedKernel
from mulearn.optimization import GurobiSolver

def get_kernel_and_solver(gram):
    eigvals = np.linalg.eigvals(gram)
    assert(sum([abs(e.imag) for e in eigvals]) < 1e-4)
    abs_neg_eigvals = [-l.real for l in eigvals if l < 0]
    adjustment = max(abs_neg_eigvals) if abs_neg_eigvals else 0

    kernel = PrecomputedKernel(gram)
    solver = GurobiSolver(adjustment=adjustment) if adjustment else GurobiSolver()

    return kernel, solver, adjustment

def get_dataset(filename):
    with open(filename) as data_file:
        data = np.array(list(csv.reader(data_file)))

    n = len(data) - 1

    # ## Extract data names, membership values and Gram matrix

    names = np.array(data[0])[1:n+1]
    mu = np.array([float(row[0]) for row in data[1:n+1]])
    gram = np.array([[float(k.replace('NA', '0')) for k in row[1:n+1]]
                     for row in data[1:n+1]])

    gram = np.delete(gram,1, axis = 0)
    gram = np.delete(gram,1, axis = 1)
    mu = np.delete(mu,1)
   

    return X, gram, mu

In [2]:
from itertools import combinations
import csv

data_file_name = 'data/data-tettamanzi-complete.csv'

with open(data_file_name) as data_file:
    data = np.array(list(csv.reader(data_file)))
    n = len(data) - 1
    names = np.array(data[0])[1:n+1]
    mu = np.array([float(row[0]) for row in data[1:n+1]])
    gram = np.array([[float(k.replace('NA', '0')) for k in row[1:n+1]]
                     for row in data[1:n+1]])
    X = np.array([x for x in np.arange(n)])
    
for i in range(250):
    comb = combinations(X, i)
    not_adjusted = 0
    adjusted = 0
    for c in list(comb):
        reducedGram0 = np.delete(gram, np.array(c), axis = 0)
        reducedGram1 = np.delete(reducedGram0, np.array(c), axis = 1)
        reducedMu = np.delete(mu,np.array(c))
        reducedX = np.delete(X,np.array(c))
        reducedX = np.array([x] for x in reducedX)
        _,_,adjustment = get_kernel_and_solver(reducedGram1)
        if adjustment:
            adjusted+=1
        else:
            not_adjusted+=1
    print(f"removing {i} formulas from gram matrix adjustment isn't necessary {not_adjusted/(adjusted + not_adjusted)*100} % of times")

D:\anaconda\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
D:\anaconda\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
D:\anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
D:\anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


removing 0 formulas from gram matrix adjustment isn't necessary 0.0 % of times
removing 1 formulas from gram matrix adjustment isn't necessary 0.0 % of times


KeyboardInterrupt: 